In [0]:
# test GPU
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
nrow = 150
ncol = 150
data_dir = '/content/drive/My Drive/Colab Notebooks/project/train'

# data_size = 2000
# folder = np.array(['Bulbasaur','Charmander','Pikachu','Squirtle'], dtype=str)

data_size = 500*12
folder = np.array(['Arcanine','Bulbasaur','Charizard','Charmander','Eevee','Jigglypuff','Lucario','Mew','Mudkip','Pikachu','Squirtle','Umbereon'], dtype=str)

In [0]:
import os
k = 0;
data_shape = (data_size,nrow,ncol)
x = np.zeros(data_shape)
y = np.zeros(data_size)
for i, j in enumerate(folder):
  class_path = os.path.join(data_dir, folder[i])
  for image_name in os.listdir(class_path):
    if(image_name != '.ipynb_checkpoints'):
      image_path = os.path.join(class_path, image_name)
      x[k,:,:] = image.load_img(image_path, target_size=(nrow, ncol), grayscale=True)
      y[k] = i
      k = k + 1

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:98: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [0]:
X = x.reshape((data_size,nrow*ncol))

In [0]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

scaler = StandardScaler()
pca = PCA()
svc = SVC(kernel='rbf')
pipe = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('svc', svc)])

In [0]:
ncomp_test = np.arange(3,11)
c = np.array([-2,-1,0,1,2,3],dtype=float)
c_test = 10**c
g = np.array([-3,-2,-1,0,1],dtype=float)
gam_test = 10**g

In [0]:
params = {'pca__n_components': ncomp_test, 'svc__C' : c_test, 'svc__gamma' : gam_test}

In [0]:
estimator = GridSearchCV(pipe, params, cv=5, return_train_score =True, iid=False)
estimator.fit(X,y)

In [0]:
print('Best test score is',estimator.best_score_)
print('Best parameters are',estimator.best_params_)

In [0]:
test_score = estimator.cv_results_['mean_test_score']
scores = test_score.reshape(len(ncomp_test),len(c_test),len(gam_test))
print(scores.shape)

i = np.where(ncomp_test==5)[0][0]
print('Index of n_components=5 is',i)

plt.imshow(scores[i, :, :])
plt.xlabel('Gamma')
plt.ylabel('C')
plt.colorbar()

ax = plt.gca()
n0 = len(c_test)
n1 = len(gam_test)
ax.set_xticks(np.arange(0,n1))
ax.set_xticklabels(gam_test)
ax.set_yticks(np.arange(0,n0))
_ = ax.set_yticklabels(c_test)
plt.show()